Script to post-process high-resolution WRF model output.

Major tasks include computing the following for selected variables:
  1. domain-averages to produce time series
  2. vertical integrals
  3. pressure-level vertical interpolation

James Ruppert  
25 Nov 2024

In [1]:
from netCDF4 import Dataset
import numpy as np
from wrf import getvar, vinterp, ALL_TIMES
import xarray as xr
from read_wrf_piccolo import *
from post_proc_functions import *
import os

In [2]:
do_2d_special = True
do_3d_vars = False
do_3d_special = False

In [3]:
########################################################
# Directories and test selection
########################################################

datdir = "/glade/derecho/scratch/ruppert/piccolo/"
# datdir = "/glade/campaign/univ/uokl0053/"

case = "sept1-4"
test_process = "ctl"
wrf_dom = "wrf_fine"
nmem = 5 # number of ensemble members

########################################################
# New functions and single-use calls go here
########################################################

# Get variable lists
vars2d = var_list_2d()
# vars3d = var_list_3d()

# Ens-member string tags (e.g., memb_01, memb_02, etc.)
memb0=1 # Starting member to read
memb_nums_str=np.arange(memb0,nmem+memb0,1).astype(str)
nustr = np.char.zfill(memb_nums_str, 2)
memb_all=np.char.add('memb_',nustr)

def memb_dir_settings(memb_dir):
    wrfdir = datdir+case+'/'+memb_dir+'/'+test_process+"/"+wrf_dom+"/"
    outdir = wrfdir+"post_proc/"
    os.makedirs(outdir, exist_ok=True)
    # Get WRF file list, dimensions
    wrffiles = get_wrf_file_list(wrfdir, "wrfout_d01*")
    # hffiles = get_wrf_file_list(wrfdir, "hfout_d01*")
    lat, lon, nx1, nx2, nz, npd = wrf_dims(wrffiles[0])
    nfiles = len(wrffiles)
    # New vertical dimension for pressure levels
    # dp = 25 # hPa
    # pres = np.arange(1000, 25, -dp)
    # nznew = len(pres)
    return outdir, wrffiles, nfiles, npd

In [4]:
if do_2d_special:

    memb_dir = memb_all[0]

    # for memb_dir in memb_all:

    # memb_dir = memb_all[comm.rank]

    print("Processing member "+memb_dir)
    outdir, wrffiles, nfiles, npd = memb_dir_settings(memb_dir)

    # Read in variable from WRF files
    # for ifile in range(nfiles):
    for ifile in range(2):

        # Open the WRF file
        file = wrffiles[ifile]
        ds = Dataset(file)

        qv = getvar(ds, "QVAPOR", timeidx=ALL_TIMES)#, cache=cache)
        pwrf = getvar(ds, "p", units='Pa', timeidx=ALL_TIMES)#, cache=cache)
        # hght = getvar(dset, "zstag", units='m', timeidx=ALL_TIMES)#, cache=cache)
        # tmpk = getvar(dset, "tk", timeidx=ALL_TIMES)#, cache=cache)
        # rho = density_moist(tmpk, qv, pwrf)

        # Get dz
        # dz = np.zeros(qv.shape)
        # for iz in range(nz):
        #     dz[:,iz] = hght[:,iz+1] - hght[:,iz]
        # Get dp
        dp = pwrf.differentiate('bottom_top')*-1

        # Process variables

        # pclass
        var = wrf_pclass(ds, dp)
        if ifile == 0:
            pclass_all = var
        else:
            pclass_all = xr.concat((pclass_all, var), 'Time')

        # pw
        var = vert_int(qv, dp)
        if ifile == 0:
            pw_all = var
        else:
            pw_all = xr.concat((pw_all, var), 'Time')

        # pw_sat
        qvsat = get_rv_sat(ds, pwrf)
        qvsat = xr.DataArray(qvsat, coords=qv.coords, dims=qv.dims, attrs=qv.attrs)
        var = vert_int(qvsat, dp)
        if ifile == 0:
            pw_sat_all = var
        else:
            pw_sat_all = xr.concat((pw_sat_all, var), 'Time')

        ds.close()

    # Remove duplicate time steps
    pclass_all = pclass_all.drop_duplicates(dim="Time", keep='first')
    pw_all     = pw_all.drop_duplicates(dim="Time", keep='first')
    pw_sat_all = pw_sat_all.drop_duplicates(dim="Time", keep='first')

    # Write out the variables
    var_name='pclass'
    write_ncfile(outdir, pclass_all, var_name)
    var_name='pw'
    write_ncfile(outdir, pw_all, var_name)
    var_name='pw_sat'
    write_ncfile(outdir, pw_sat_all, var_name)

print("Done writing out special 2D variables")

Processing member memb_01


: 